In [17]:
from ldpc.codes import rep_code
from bposd.hgp import hgp
import numpy as np
from ldpc import bposd_decoder
from scipy.sparse import coo_matrix
from mqt.qecc import *
import time
from tqdm import tqdm

# 参数设置

# # [72,12,6]
# ell = 6
# m = 6

# # [90,8,10]
# ell = 15
# m = 3

# # [108,8,10]
# ell = 9
# m = 6

# [144,12,12]
# ell = 12
# m = 6

# # [288,12,18]
ell = 12
m = 12

# # [784,24,24]
# ell = 28
# m = 14

n = 2 * m * ell


# 定义循环移位矩阵
def cyclic_shift_matrix(size, shift):
    I = np.eye(size, dtype=int)
    return np.roll(I, shift, axis=1)


# 生成多项式 A 和 B 的循环矩阵
A1 = cyclic_shift_matrix(ell * m, 3)  # x^3
A2 = cyclic_shift_matrix(ell * m, 1)  # y^1
A3 = cyclic_shift_matrix(ell * m, 2)  # y^2
A = (A1 + A2 + A3) % 2  # A = x^3 + y^1 + y^2

B1 = cyclic_shift_matrix(ell * m, 3)  # y^3
B2 = cyclic_shift_matrix(ell * m, 1)  # x^1
B3 = cyclic_shift_matrix(ell * m, 2)  # x^2
B = (B1 + B2 + B3) % 2  # B = y^3 + x^1 + x^2

# 生成校验矩阵 H
Hx = np.hstack((A, B))
Hz = np.hstack((B.T, A.T))

# 打印校验矩阵 Hx 和 Hz
# print("Hx (X-check matrix):")
# print(Hx)
# print("\nHz (Z-check matrix):")
# print(Hz)
# print("Hx shape:", Hx.shape)

Hx shape: (144, 288)


AC Decoder

In [12]:
import numpy as np


# Belief Propagation (BP) Implementation
def belief_propagation(H, sigma, max_iterations=10):
    """
    Belief Propagation to estimate posterior probabilities.
    H: Parity-check matrix
    sigma: Syndrome
    max_iterations: Maximum number of BP iterations
    """
    m, n = H.shape
    messages = np.zeros((m, n))  # Messages passed between variable and check nodes
    posteriors = np.full(n, 0.5)  # Initialize posterior probabilities

    for iteration in range(max_iterations):
        # Variable to Check node updates
        for i in range(m):
            for j in range(n):
                if H[i, j] == 1:
                    product = np.prod(
                        [
                            1 - 2 * messages[i, k]
                            for k in range(n)
                            if H[i, k] == 1 and k != j
                        ]
                    )
                    messages[i, j] = (1 - sigma[i]) * 0.5 * (1 + product) + sigma[
                        i
                    ] * 0.5 * (1 - product)

        # Check to Variable node updates
        for j in range(n):
            product = 1
            for i in range(m):
                if H[i, j] == 1:
                    product *= 1 - 2 * messages[i, j]
            posteriors[j] = 0.5 * (1 + product)

    return posteriors


# Stage 2: Cluster Formation
def cluster_formation(H, posteriors, K):
    """
    Cluster Formation based on posteriors.
    H: Parity-check matrix
    posteriors: Posterior probabilities from BP
    K: Number of columns to select for clusters
    """
    # Sort columns based on posterior probabilities
    column_order = np.argsort(-posteriors)  # Descending order
    selected_columns = column_order[:K]

    # Form clusters using Gaussian elimination
    clusters = []
    for col in selected_columns:
        # Perform pivot operation and form a new cluster
        cluster = [col]
        for i in range(H.shape[0]):
            if H[i, col] == 1:
                cluster.extend(np.where(H[i] == 1)[0])
        clusters.append(set(cluster))

    return clusters


# Stage 3: Decode Each Cluster
def decode_clusters(H, sigma, clusters):
    """
    Decode errors within each cluster independently.
    H: Parity-check matrix
    sigma: Syndrome
    clusters: List of clusters
    """
    decoded_errors = np.zeros(H.shape[1], dtype=int)

    for cluster in clusters:
        sub_H = H[:, list(cluster)]
        sub_sigma = sigma
        try:
            # Solve for errors in the cluster
            error = np.linalg.solve(sub_H, sub_sigma)
            for idx, col in enumerate(cluster):
                decoded_errors[col] = error[idx]
        except np.linalg.LinAlgError:
            # If no solution, continue to the next cluster
            pass

    return decoded_errors


# AC Main Function
def ambiguity_clustering(H, sigma, max_iterations=10, K=10):
    """
    Main function to perform AC decoding.
    H: Parity-check matrix
    sigma: Syndrome
    max_iterations: Maximum number of BP iterations
    K: Number of columns to select for clusters
    """
    # Step 1: Belief Propagation
    posteriors = belief_propagation(H, sigma, max_iterations)

    # Step 2: Cluster Formation
    clusters = cluster_formation(H, posteriors, K)

    # Step 3: Decode Each Cluster
    decoded_errors = decode_clusters(H, sigma, clusters)

    return decoded_errors


# Parameters for AC
max_iterations = 9  # BP iterations
K = int(0.1 * n) / 3  # The number of additional columns added to C in stage 2

In [14]:
# UF decoder
code = Code(Hx, Hz)
uf_decoder = UFHeuristic()
uf_decoder.set_code(code)

# BP decoder
Hx_sparse = coo_matrix(Hx)
bpx_decoder = bposd_decoder(
    Hx_sparse,
    channel_probs=[None],
    max_iter=10000,
    bp_method="ms",
    ms_scaling_factor=0,
    osd_method="osd_cs",
    osd_order=7,
)

trials = 100

good_trials_bp = 0
bad_trials_bp = 0
good_trials_uf = 0
bad_trials_uf = 0
good_trials_ac = 0
bad_trials_ac = 0

time_uf = 0
time_bp = 0
time_ac = 0

for i in tqdm(range(trials), desc="Running trials"):
    x_error = sample_iid_pauli_err(code.n, 0.05)
    x_syndrome = code.get_x_syndrome(x_error)

    """
    UF decoder
    """
    uf_start_time = time.time()
    uf_decoder.decode(x_syndrome)
    uf_end_time = time.time()
    result = uf_decoder.result
    residual_err_uf = np.array(x_error) ^ np.array(result.estimate)
    if code.is_x_stabilizer(residual_err_uf):
        good_trials_uf += 1
    else:
        bad_trials_uf += 1
    time_uf += uf_end_time - uf_start_time

    """
    BP+OSD decoder
    """
    x_syndrome_01 = np.array([int(x) for x in x_syndrome]).flatten()
    bp_start_time = time.time()
    bpx_decoder.decode(x_syndrome_01)
    bp_end_time = time.time()
    low_weight_error = bpx_decoder.osdw_decoding

    # 计算残余错误
    residual_err_bposd = (low_weight_error + x_error) % 2
    if code.is_x_stabilizer(residual_err_bposd):  # 使用 code.is_x_stabilizer
        good_trials_bp += 1
    else:
        bad_trials_bp += 1
    time_bp += bp_end_time - bp_start_time

    """
    AC decoder
    """
    ac_start_time = time.time()
    ac_result = ambiguity_clustering(Hx, x_syndrome, max_iterations, K)
    ac_end_time = time.time()
    residual_err_ac = (ac_result + x_error) % 2
    if code.is_x_stabilizer(residual_err_ac):  # 使用 code.is_x_stabilizer
        good_trials_ac += 1
    else:
        bad_trials_ac += 1
    time_ac += ac_end_time - ac_start_time


print(f"good_trials_uf = {good_trials_uf}")
print(f"bad_trials_uf = {bad_trials_uf}")

print(f"good_trials_bp = {good_trials_bp}")
print(f"bad_trials_bp = {bad_trials_bp}")

print(f"good_trials_ac = {good_trials_ac}")
print(f"bad_trials_ac = {bad_trials_ac}")

print(f"average time for UF = {time_uf/trials}")
print(f"average time for BP+OSD = {time_bp/trials}")
print(f"average time for AC = {time_ac/trials}")

Running trials: 100%|██████████| 100/100 [00:15<00:00,  6.41it/s]

good_trials_uf = 0
bad_trials_uf = 100
good_trials_bp = 0
bad_trials_bp = 100
good_trials_ac = 0
bad_trials_ac = 100
average time for UF = 5.652189254760742e-05
average time for BP+OSD = 0.0551467752456665
average time for AC = 0.09893503665924072
